In [1]:
import numpy
from sortedcontainers import SortedDict
import math

In [2]:
# Global Parameters:
t = 0.0 #start at time 0
t_max = 12 #2 months in hrs (assume no nights so 12 hr = 1 day)
timeholder = 0 
#timeline = [] #old way of initializing the timeline
timeline = SortedDict() #New and improved timeline (using a sorted dictionary)!
discards = [] #where we put the old version of tickets that get over-written/moved
#We've moved away from using these params with expovariate (how we handle FV is still TBD)
#So THE BELOW 3 SHOULD BE COMMENTED OUT (but rn we don't want the handler funcs to yell at us)
FV_param = [0, t_max] #5days, tmax/3 #0.2 #males/0.19 #totally random
FG_param = 3 #larger values lead to foraging ending earlier, because taus are smaller
SB_param = 1.5 #larger values lead to bowerstay ending earlier, because taus are smaller
RB_param = 6 #made up -- takes 30mins (every time -- no distribution) to repair bower
MA_param = 0.1 #made up -- takes 6mins (every time -- no distribution) to maraud bower
MT_vs_FG = 0.1 #.05 originally #compared 20 feedings/day to .1 maraudings/hr
males = 12
next_forage = [0] * males #NOTE- later we'll generate the initial next_forage times in the deepest part of our while loop (so for each simulation the first foraging times are unique)
bower_states = [0] * males
male_states = [0] * males 
mating_states = [0] * males
fitness_states = numpy.array([0] * males)
success_rate = 1
success_times = [] #going to change this to be a vector of times when the female leaves (at this point, sucessful mating has occured)
recents_list = []
max_visits = 6 #STEEEEVEEEE
bird_speed = 12 * 3600 #m/hr (12m/s)
d = 1000.0
FperM = 3 #number of sexually mature females per sexually mature males in a bowerbird population
num_sims = 1
not_maraud_time = 0.15
nodes=12


In [3]:
def envgenerator(nodes, x_dim, y_dim):
    Xs = numpy.random.rand(nodes) * x_dim
    Ys = numpy.random.rand(nodes) * y_dim
    return [Xs,Ys]

def dists(xy, bird_speed):
    node_dist = numpy.zeros(nodes * nodes).reshape((nodes, nodes))
    travel_times = numpy.zeros(nodes * nodes).reshape((nodes, nodes))
    for i in range(nodes):
        for j in range(i + 1, nodes):
            dist = math.sqrt((xy[0][j] - xy[0][i]) ** 2 + (xy[1][j] - xy[1][i]) ** 2)
            travel = dist / bird_speed
            node_dist[j][i] = dist
            node_dist[i][j] = dist
            travel_times[j][i] = travel
            travel_times[i][j] = travel
    return [node_dist,travel_times]

xy = envgenerator(nodes, 400, 500)

thing=dists(xy, bird_speed)
travel_times=thing[1]

In [4]:
timeline = SortedDict()

In [5]:
def ticketgenerator(t, tau, o, a, targ, t2=0): #=0 is a quick fix only
    ticket = {
        'time': t, #time at which the activity starts
        'tau': tau, #how long the activity lasts
        'owner': o, #who is doing the activity (could be a female if action=-3)
        'action': a, #which activity (see key)
        'target': targ, #target of the activity (owner=target unless action=-3,-4,-5, or -6)
        'origin': t2 #the time of the ticket that generated this one
    }
    return ticket

In [6]:
# try adding tickets to timeline
def addtotimeline(ticket):
    global timeline
    timeline[ticket["time"]] = ticket

In [7]:
def nexttau(action, ow, targ):
    new_tau=-1
    while new_tau<=0:
        switcher = { #NOTE: used abs
            -1: numpy.random.normal(loc=.1583, scale=.09755, size=1)[0], #choose when to leave bower (generate a tau for bower stay)
            -2: numpy.random.gamma(shape=1.5, scale=5, size=1)[0]/60, #choose when to stop foraging (generate a tau for foraging)
            -3: numpy.random.uniform(FV_param[0], FV_param[1]), #FV_param... totally arbitrary so we should think about it
            -4: travel_times[ow][targ],
            -5: MA_param, #in the future we'll do something with it
            -6: travel_times[targ][ow],
            -7: 1 / (1-MT_vs_FG) * (numpy.random.normal(loc=.1583, scale=.09755, size=1)[0]) #rescale SB_param because no marauding can occur from RB (want FG times to be comparable)
        }
        new_tau=switcher.get(action)
    return new_tau


In [8]:
#initialize the timeline
tmp = ticketgenerator(.1, .1, 10, -4, 9, 1)
addtotimeline(tmp)
print(timeline)
tmp1 = ticketgenerator(.15, .15, 11, -4, 6, 2)
addtotimeline(tmp1)
print(timeline)

SortedDict({0.1: {'time': 0.1, 'tau': 0.1, 'owner': 10, 'action': -4, 'target': 9, 'origin': 1}})
SortedDict({0.1: {'time': 0.1, 'tau': 0.1, 'owner': 10, 'action': -4, 'target': 9, 'origin': 1}, 0.15: {'time': 0.15, 'tau': 0.15, 'owner': 11, 'action': -4, 'target': 6, 'origin': 2}})


In [9]:
bower_states = [0] * 12
male_states = [100] * 12

In [10]:
def statechanger(new_tic):
    options = {-1: SBstatechanger,
               -2: FGstatechanger,
               -3: FVstatechanger,
               -4: MTstatechanger,
               -5: MAstatechanger,
               -6: MRstatenotchanger,
               -7: RBstatechanger
              }
    options[new_tic['action']](new_tic)
    
def SBstatechanger(new_tic): #tech anger
    ow=new_tic['owner']
    t=new_tic['time']
    male_states[ow]=t #male is at his bower / store time so that we FV can access and modify SB_tic in sorted dict

def FGstatechanger(new_tic):
    ow=new_tic['owner']
    male_states[ow]=0

def FVstatechanger(new_tic): #here the t is the start time... idea for future: female start and end tics?
    targ=new_tic['target']
    t=new_tic['time']
    if bower_states[targ]==0 and male_states[targ]!=0 and mating_states[targ]<t:
        fitness_states[targ]+=1
        mating_end=.25+t #can use dist instead of .25
        if t<mating_end:
            SB_tic=tl_dict[t] #PSEUDO
            SB_tic['tau']=SB_tic['tau']+(mating_end-SB_tic['time'])
            SB_tic['time']=mating_end
            #re-add tic to sorted dict (if not done automatically)
        mating_states[targ]=mating_end      
    
def MTstatechanger(new_tic):
    ow=new_tic['owner']
    male_states[ow]=0

def MAstatechanger(new_tic): #store number of successful marauds of each male --> MAKE A NEW STATE
    targ=new_tic['target']
    if bower_states[targ]==0 and male_states[targ]==0:
        bower_states[targ]=-1*RB_param
    #MAKE IT EITHER TAKE maraud_time or 0 (in the else)

def MRstatenotchanger(new_tic):
    print(" ")

def RBstatechanger(new_tic):
    tau=new_tic['tau']
    if bower_states[ow]+tau>0:
        new_tic['time']-=bower_states[ow]+tau #think about it more in the AM :)
        tau=-1*bower_states[ow]
        new_tic['tau']=tau
        #add changes to timeline dict if necessary
    bower_states[ow]+=tau
    male_states[ow]=new_tic['time'] #although it doesn't matter from an FV POV

def futurebuilder(old_tic, new_ac, new_targ):
    old_t=old_tic['time']
    new_tic={}
    if math.isclose(old_t, t_max, rel_tol=0, abs_tol=.0000001)==0: #if old_t is not essentially t_max
        ow=old_tic['owner']
        new_tau=nexttau(new_ac, ow, new_targ)
        end_t=old_t+new_tau #this time is the time when the action has ended
        if end_t>t_max: 
            new_tau=new_tau-(end_t-t_max) #modify the new_tau so end_t will be t_max
            end_t=old_t+new_tau #recalculate end_t
        new_tic=ticketgenerator(end_t, new_tau, ow, new_ac, new_targ) #now the time represents the end time for the action
    return new_tic

print(timeline)
for key in timeline:
    tic=timeline[key]
    ac=tic['action']
    ow=tic['owner']
    targ=tic['target']
    if ac==-4:
        #generate a -5 tic as new_tic and pass it in
        new_tic=futurebuilder(tic, -5, targ)
    elif ac==-5:
        #dis[1]/use travel times.. but this is just an example
        new_tic=futurebuilder(tic, -6, targ)
    else:
        print(" ")
    if new_tic=={}:
        print("done (for some owner!")
    else:
        statechanger(new_tic)
        addtotimeline(new_tic)
        #t=next_tic['time'] #new time based on the ticket we just read
        #timeholder=timeholder+1 #increment timeholder 



SortedDict({0.1: {'time': 0.1, 'tau': 0.1, 'owner': 10, 'action': -4, 'target': 9, 'origin': 1}, 0.15: {'time': 0.15, 'tau': 0.15, 'owner': 11, 'action': -4, 'target': 6, 'origin': 2}})
 
 
 
 
 
 


In [11]:
print(timeline)


SortedDict({0.1: {'time': 0.1, 'tau': 0.1, 'owner': 10, 'action': -4, 'target': 9, 'origin': 1}, 0.15: {'time': 0.15, 'tau': 0.15, 'owner': 11, 'action': -4, 'target': 6, 'origin': 2}, 0.2: {'time': 0.2, 'tau': 0.1, 'owner': 10, 'action': -5, 'target': 9, 'origin': 0}, 0.20457633659992652: {'time': 0.20457633659992652, 'tau': 0.0045763365999265, 'owner': 10, 'action': -6, 'target': 9, 'origin': 0}, 0.25: {'time': 0.25, 'tau': 0.1, 'owner': 11, 'action': -5, 'target': 6, 'origin': 0}, 0.2535412520772618: {'time': 0.2535412520772618, 'tau': 0.003541252077261799, 'owner': 11, 'action': -6, 'target': 6, 'origin': 0}})
